
# Parkinson Disease Detector according to risk factors

In [1]:
import pandas as pd

# CSV
raw_url = "https://raw.githubusercontent.com/sharbt/PDDetector/refs/heads/main/pd_dataset.csv"
pdf = pd.read_csv(raw_url)

# Convert to Spark DataFrame
df = spark.createDataFrame(pdf)
df.show()
    


StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 3, Finished, Available, Finished)

+---------+---+------+--------------+---------------+-----------------+-----------------+------------+----------------+-------------+------------+-------------------+-------------------+
|PatientID|Age|Gender|SmokingHistory|FamilyHistoryPD|HeadInjuryHistory|PesticideExposure|CoffeeIntake|PhysicalActivity|OlfactoryLoss|Constipation|REMBehaviorDisorder|ParkinsonsDiagnosis|
+---------+---+------+--------------+---------------+-----------------+-----------------+------------+----------------+-------------+------------+-------------------+-------------------+
|        1| 67|  Male|        Former|              0|                0|                0|         Low|            High|            1|           0|                  1|                  1|
|        2| 69|Female|         Never|              0|                0|                0|    Moderate|        Moderate|            0|           0|                  0|                  0|
|        3| 58|  Male|        Former|              1|            

In [2]:
# Code generated by Data Wrangler for pandas DataFrame

import pandas as pd

def clean_data(pdf):
    # One-hot encode column: 'SmokingHistory'
    insert_loc = pdf.columns.get_loc('SmokingHistory')
    pdf = pd.concat([pdf.iloc[:,:insert_loc], pd.get_dummies(pdf.loc[:, ['SmokingHistory']]), pdf.iloc[:,insert_loc+1:]], axis=1)
    # One-hot encode column: 'CoffeeIntake'
    insert_loc = pdf.columns.get_loc('CoffeeIntake')
    pdf = pd.concat([pdf.iloc[:,:insert_loc], pd.get_dummies(pdf.loc[:, ['CoffeeIntake']]), pdf.iloc[:,insert_loc+1:]], axis=1)
    # One-hot encode column: 'Gender'
    insert_loc = pdf.columns.get_loc('Gender')
    pdf = pd.concat([pdf.iloc[:,:insert_loc], pd.get_dummies(pdf.loc[:, ['Gender']]), pdf.iloc[:,insert_loc+1:]], axis=1)
    # One-hot encode column: 'PhysicalActivity'
    insert_loc = pdf.columns.get_loc('PhysicalActivity')
    pdf = pd.concat([pdf.iloc[:,:insert_loc], pd.get_dummies(pdf.loc[:, ['PhysicalActivity']]), pdf.iloc[:,insert_loc+1:]], axis=1)
    # Drop column: 'PatientID'
    pdf = pdf.drop(columns=['PatientID'])
    # Drop duplicate rows across all columns
    pdf = pdf.drop_duplicates()
    return pdf

pdf_clean = clean_data(pdf.copy())
display(pdf_clean)

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 32e571e9-b569-41b7-9e44-5680af299ae2)

In [3]:
from sklearn.model_selection import train_test_split
    
X, y = pdf_clean[['Age','Gender_Female', 'Gender_Male',
'SmokingHistory_Current', 'SmokingHistory_Former', 'SmokingHistory_Never' , \
'FamilyHistoryPD','HeadInjuryHistory', 'PesticideExposure', \
'CoffeeIntake_High', 'CoffeeIntake_Low', 'CoffeeIntake_Moderate', \
'PhysicalActivity_High','PhysicalActivity_Low','PhysicalActivity_Moderate' , \
'OlfactoryLoss','Constipation', \
'REMBehaviorDisorder']].values, pdf_clean['ParkinsonsDiagnosis'].values
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)




StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 5, Finished, Available, Finished)

In [4]:
import mlflow
experiment_name = "pd-classification"
mlflow.set_experiment(experiment_name)


StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 6, Finished, Available, Finished)

2025/08/20 15:07:52 INFO mlflow.tracking.fluent: Experiment with name 'pd-classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='sds://onelakeuksouth.pbidedicated.windows.net/55404d16-8157-4838-9ac8-ae026de15213/4f0725e3-391c-415b-9b51-a311992325e9', creation_time=1755702477239, experiment_id='4f0725e3-391c-415b-9b51-a311992325e9', last_update_time=1755702477239, lifecycle_stage='active', name='pd-classification', tags={}>

In [5]:
from sklearn.linear_model import LogisticRegression
    
with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = LogisticRegression(C=1/0.1, solver="liblinear")
    model.fit(X_train, y_train)

    mlflow.log_param("estimator", "LogisticRegression")

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 7, Finished, Available, Finished)

2025/08/20 15:08:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [6]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 8, Finished, Available, Finished)

Accuracy: 0.8832929782082325
              precision    recall  f1-score   support

           0       0.90      0.91      0.90      1234
           1       0.86      0.85      0.85       831

    accuracy                           0.88      2065
   macro avg       0.88      0.88      0.88      2065
weighted avg       0.88      0.88      0.88      2065



In [7]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    mlflow.log_param("estimator", "RandomForestClassifier")

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 9, Finished, Available, Finished)

In [8]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 10, Finished, Available, Finished)

Accuracy: 0.9167070217917676
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1234
           1       0.91      0.89      0.90       831

    accuracy                           0.92      2065
   macro avg       0.91      0.91      0.91      2065
weighted avg       0.92      0.92      0.92      2065



In [9]:
import mlflow
experiments = mlflow.search_experiments()
for exp in experiments:
    print(exp.name)

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 11, Finished, Available, Finished)

pd-classification


In [10]:
experiment_name = "pd-classification"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 12, Finished, Available, Finished)

<Experiment: artifact_location='sds://onelakeuksouth.pbidedicated.windows.net/55404d16-8157-4838-9ac8-ae026de15213/4f0725e3-391c-415b-9b51-a311992325e9', creation_time=1755702477239, experiment_id='4f0725e3-391c-415b-9b51-a311992325e9', last_update_time=1755702477239, lifecycle_stage='active', name='pd-classification', tags={}>


In [11]:
mlflow.search_runs(exp.experiment_id)

StatementMeta(, 0febfd10-cc92-4f98-8e86-ce617de1e500, 13, Finished, Available, Finished)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_f1_score,metrics.training_roc_auc,metrics.training_recall_score,metrics.training_accuracy_score,...,tags.estimator_class,tags.synapseml.run.artifactJobId,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.estimator_name,tags.synapseml.livy.id,tags.synapseml.user.name,tags.mlflow.source.type,tags.synapseml.user.id
0,76867bca-3a3a-4ea9-9e19-2bdc318a5927,4f0725e3-391c-415b-9b51-a311992325e9,FINISHED,sds://onelakeuksouth.pbidedicated.windows.net/...,2025-08-20 15:08:26.349000+00:00,2025-08-20 15:08:31.300000+00:00,0.994394,0.999935,0.994394,0.994394,...,sklearn.ensemble._forest.RandomForestClassifier,fe298fae-d380-4cfa-bb81-16ef50a8fd98,/home/trusted-service-user/cluster-env/trident...,trusted-service-user,helpful_vulture_wv6ydb6b,RandomForestClassifier,0febfd10-cc92-4f98-8e86-ce617de1e500,Sharanu Turamari,LOCAL,fee6b27d-9c0d-4398-abdc-d0794ebb9688
1,131120aa-9deb-4929-a485-c92676ad2191,4f0725e3-391c-415b-9b51-a311992325e9,FINISHED,sds://onelakeuksouth.pbidedicated.windows.net/...,2025-08-20 15:08:01.596000+00:00,2025-08-20 15:08:23.105000+00:00,0.883138,0.959810,0.883306,0.883306,...,sklearn.linear_model._logistic.LogisticRegression,a0371ce9-4116-4a5e-8bd3-1cd92d7dcb7a,/home/trusted-service-user/cluster-env/trident...,trusted-service-user,musing_card_510031n2,LogisticRegression,0febfd10-cc92-4f98-8e86-ce617de1e500,Sharanu Turamari,LOCAL,fee6b27d-9c0d-4398-abdc-d0794ebb9688
